# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import hvplot
import hvplot.pandas
import requests
import geopandas as gpd
# import geopandas as gpd
# from geopandas import GeoDataFrame
from shapely.geometry import Point
# import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,21.69,98,40,3.09,US,1698595848
1,1,tobelo,1.7284,128.0095,26.55,79,86,0.38,ID,1698595849
2,2,edinburgh of the seven seas,-37.0676,-12.3116,11.23,66,21,10.66,SH,1698595849
3,3,port-aux-francais,-49.3500,70.2167,2.32,98,100,7.87,TF,1698595850
4,4,stanley,54.8680,-1.6985,9.33,95,75,3.09,GB,1698595850


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
geometry = [Point(xy) for xy in zip(city_data_df.Lng, city_data_df.Lat)]
geo_city_df = gpd.GeoDataFrame(city_data_df, crs="EPSG:4326", geometry=geometry) 

# Display the map
geo_city_df.hvplot(s="Humidity",c="City", global_extent=True, frame_height=450, tiles=True)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
no_clouds= city_data_df[city_data_df["Cloudiness"] ==0]
no_cold= no_clouds[(no_clouds["Max Temp"] > 10)] 
no_hot= no_cold[(no_cold["Max Temp"] < 27)] 
slow_wind= no_hot[(no_hot["Wind Speed"] < 4)]
ideal_weather = slow_wind

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,cabo san lucas,22.8909,-109.9124,26.17,69,0,3.60,MX,1698595868
96,96,haiku-pauwela,20.9219,-156.3051,23.91,77,0,3.60,US,1698595904
111,111,shingu,33.7333,135.9833,11.20,81,0,2.92,JP,1698595911
172,172,swainsboro,32.5974,-82.3337,26.03,59,0,1.83,US,1698595947
213,213,creel,27.7500,-107.6333,14.91,23,0,2.50,MX,1698595972
228,228,corovode,40.5042,20.2272,18.14,68,0,1.69,AL,1698595979
231,231,zadar,44.1197,15.2422,19.34,88,0,3.60,HR,1698595980
234,234,karachi,24.9056,67.0822,26.90,61,0,2.57,PK,1698595960
308,308,ji'an,27.1172,114.9793,19.56,79,0,1.43,CN,1698596055
335,335,loushanguan,28.1375,106.8200,14.27,86,0,0.32,CN,1698596071


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# hotel_df= pd.DataFrame.copy(ideal_weather)
hotel_df = ideal_weather.copy()[["City", "Country", "Lat", "Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,cabo san lucas,MX,22.8909,-109.9124,69,
96,haiku-pauwela,US,20.9219,-156.3051,77,
111,shingu,JP,33.7333,135.9833,81,
172,swainsboro,US,32.5974,-82.3337,59,
213,creel,MX,27.7500,-107.6333,23,
228,corovode,AL,40.5042,20.2272,68,
231,zadar,HR,44.1197,15.2422,88,
234,karachi,PK,24.9056,67.0822,61,
308,ji'an,CN,27.1172,114.9793,79,
335,loushanguan,CN,28.1375,106.8200,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = "10000"
params = {
    "categories": ["accommodation.hotel"], 
    "apiKey": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    Latitude =str(row["Lat"])
    Longitude = str(row["Lng"])

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:" + Longitude + "," + Latitude + "," + radius

    params["bias"] = "proximity:" + Longitude + "," + Latitude

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
shingu - nearest hotel: ホテルニューパレス
swainsboro - nearest hotel: No hotel found
creel - nearest hotel: Hotel Cascada inn
corovode - nearest hotel: Bar Hotel Sokoli
zadar - nearest hotel: No hotel found
karachi - nearest hotel: Quetta Agha Hotel
ji'an - nearest hotel: 华侨国际酒店
loushanguan - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
cartersville - nearest hotel: Quality Inn & Suites
huai'an - nearest hotel: No hotel found
atlantic - nearest hotel: Izzy’s Steak & Seafood House at The Inn at Sugar Hill


,City,Country,Lat,Lng,Humidity,Hotel Name
36,cabo san lucas,MX,22.8909,-109.9124,69,Comfort Rooms
96,haiku-pauwela,US,20.9219,-156.3051,77,Inn At Mama's Fish House
111,shingu,JP,33.7333,135.9833,81,ホテルニューパレス
172,swainsboro,US,32.5974,-82.3337,59,No hotel found
213,creel,MX,27.7500,-107.6333,23,Hotel Cascada inn
228,corovode,AL,40.5042,20.2272,68,Bar Hotel Sokoli
231,zadar,HR,44.1197,15.2422,88,No hotel found
234,karachi,PK,24.9056,67.0822,61,Quetta Agha Hotel
308,ji'an,CN,27.1172,114.9793,79,华侨国际酒店
335,loushanguan,CN,28.1375,106.8200,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
#%%capture --no-display

# Configure the map plot
geometry = [Point(xy) for xy in zip(hotel_df.Lng, hotel_df.Lat)]
geo_hotel_df = gpd.GeoDataFrame(hotel_df, crs="EPSG:4326", geometry=geometry) 

# Display the map
geo_hotel_df.hvplot(s="Humidity",c="City", global_extent=True, frame_height=450, tiles=True, hover_cols=["Country", "Hotel Name"])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)